# LlamaIndexを用いて、クイズ生成を考える

- インデックスを作成する
- BeautifulSoupをつかって記事本文だけをインデックス
- 解答候補を抽出する
- 関連情報を得る
- 質問生成：四択問題
- 検証: 質問＋各選択肢が成立するか
- 導入文の生成
- 導入文と質問・解答をペアにしてすわりがよいかを評価
- クイズの難しさの評価
- クイズとしての面白さの評価







- [ChatGPTで独自データを利用できるLlamaIndexはどんな仕組みで動いているのか？調べてみました | DevelopersIO](https://dev.classmethod.jp/articles/llamaindex-overview/)

## 対象の記事

In [304]:
# この例では、この記事を使います

article_urls=['https://nordot.app/1014472568104288256']  # 細野勇策が通算16アンダー首位　東建男子ゴルフ第2日 ｜ 共同通信 (2023/03/31)

In [302]:
# nordotから一覧

from urllib import request  # urllib.requestモジュールをインポート
import json

url = 'https://nordot.app/-/units/39166665832988672/list?offset=10&limit=10'
user_agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'   # User-Agentを設定しないと応答を返さない
headers = {'User-Agent': user_agent}

req = request.Request(url, headers=headers)
with request.urlopen(req) as response:
   content = json.loads(response.read().decode('utf8'))


[('NY円、132円台後半', 'https://nordot.app/1014521617494933504'),
 ('トランプ氏、4日にも出頭', 'https://nordot.app/1014495251956924416'),
 ('不正閲覧、改善命令を勧告', 'https://nordot.app/1014499439336554496'),
 ('西2―3オ（31日）', 'https://nordot.app/1014513667462299648'),
 ('WBC優勝の熱気つなげる', 'https://nordot.app/1014518304881295360'),
 ('前大統領起訴「ノーコメント」とバイデン氏', 'https://nordot.app/1014518711919214592'),
 ('厚労省、「エホバ」と面会し要請', 'https://nordot.app/1014499176452636672'),
 ('神6―3D（31日）', 'https://nordot.app/1014518178649620480'),
 ('米国との戦略核の均衡保つとロ外交政策概念', 'https://nordot.app/1014517638885523456'),
 ('NY円、132円82～92銭', 'https://nordot.app/1014517200072998912')]

In [303]:
[
    (p["title"], p["published_at"], f'https://nordot.app/{p["id"]}') for p in content['posts']
]


[('NY円、132円台後半',
  '2023-03-31T13:06:07+00:00',
  'https://nordot.app/1014521617494933504'),
 ('トランプ氏、4日にも出頭',
  '2023-03-31T13:05:53+00:00',
  'https://nordot.app/1014495251956924416'),
 ('不正閲覧、改善命令を勧告',
  '2023-03-31T12:58:56+00:00',
  'https://nordot.app/1014499439336554496'),
 ('西2―3オ（31日）',
  '2023-03-31T12:56:03+00:00',
  'https://nordot.app/1014513667462299648'),
 ('WBC優勝の熱気つなげる',
  '2023-03-31T12:55:41+00:00',
  'https://nordot.app/1014518304881295360'),
 ('前大統領起訴「ノーコメント」とバイデン氏',
  '2023-03-31T12:54:48+00:00',
  'https://nordot.app/1014518711919214592'),
 ('厚労省、「エホバ」と面会し要請',
  '2023-03-31T12:54:37+00:00',
  'https://nordot.app/1014499176452636672'),
 ('神6―3D（31日）',
  '2023-03-31T12:54:24+00:00',
  'https://nordot.app/1014518178649620480'),
 ('米国との戦略核の均衡保つとロ外交政策概念',
  '2023-03-31T12:50:41+00:00',
  'https://nordot.app/1014517638885523456'),
 ('NY円、132円82～92銭',
  '2023-03-31T12:48:11+00:00',
  'https://nordot.app/1014517200072998912')]

## インデックスを作成する


In [239]:
!pip install -qU llama-index wikipedia langchain html2text

In [240]:
!pip install -qU gpt-index

In [241]:
import os
os.environ["OPENAI_API_KEY"] = 'YOUR_OPENAI_API_KEY'


from llama_index import GPTSimpleVectorIndex, SimpleDirectoryReader
#documents = SimpleDirectoryReader('data').load_data()
#index = GPTSimpleVectorIndex.from_documents(documents)

In [242]:
from llama_index import GPTSimpleVectorIndex, SimpleWebPageReader

documents = SimpleWebPageReader().load_data(article_urls)
index = GPTSimpleVectorIndex.from_documents(documents)

In [243]:
index.query("<question_text>?")

Response(response='\nThe answer to the question is not provided in the context information.', source_nodes=[NodeWithScore(node=Node(text='   <path class="cls-1" d="M170.46,63.44A22.18,22.18,0,0,1,162,61.77a20.89,20.89,0,0,1-7.25-5,24.69,24.69,0,0,1-5.06-8.08,29.6,29.6,0,0,1-1.91-11v-.18a29.6,29.6,0,0,1,1.91-11,25.27,25.27,0,0,1,5-8.08,20.47,20.47,0,0,1,7.2-5,22.3,22.3,0,0,1,8.55-1.67A17.68,17.68,0,0,1,179.79,14a24.73,24.73,0,0,1,6.18,5.2v-18A1.25,1.25,0,0,1,187.21,0h11.64a1.25,1.25,0,0,1,1.24,1.24v60a1.25,1.25,0,0,1-1.24,1.24H187.21A1.25,1.25,0,0,1,186,61.27V55.36a22.2,22.2,0,0,1-6.27,5.8A17.24,17.24,0,0,1,170.46,63.44Zm3.62-12a11.36,11.36,0,0,0,4.69-1,11.22,11.22,0,0,0,3.81-2.78,14,14,0,0,0,2.6-4.37,15.64,15.64,0,0,0,1-5.62v-.18a15.6,15.6,0,0,0-1-5.62,13.89,13.89,0,0,0-2.6-4.37,11.12,11.12,0,0,0-3.81-2.79,11.82,11.82,0,0,0-9.38,0,11.56,11.56,0,0,0-3.85,2.79,13.47,13.47,0,0,0-2.65,4.32,15.53,15.53,0,0,0-1,5.67v.18a15.64,15.64,0,0,0,1,5.62,13.39,13.39,0,0,0,2.65,4.37,11.67,11.67,0,0,0,3

In [244]:
index.query("次の問いに日本語で答えてください。単語で出力してください。\n\n誰が首位?").response.strip()

'細野勇策が共同通信の首位にいる'

In [245]:
index.query("要約してください").response.strip()

'このウェブページは、20歳の細野勇策が東建ホームメイト・カップ第2日で16アンダー首位を獲得したことを報じる共同通信の記事であり、ヘッダーに共同通信のロゴ画像が表示されています。また、Twitterで共同通信をフォローすることもできます。'

## BeautifulSoupをつかって記事本文だけをインデックス

こちらの方がhtmlタグなどのノイズが少ない。きれいな情報を取得でき、それを使った結果もよさそう。

In [246]:
!pip install -qU bs4

In [247]:
from urllib import request
from bs4 import BeautifulSoup

url = 'https://nordot.app/1014495251956924416'

url = article_urls[0]
response = request.urlopen(url)
soup = BeautifulSoup(response)
response.close()

In [248]:
text = soup.find('div', class_='main__articleBody').text.strip().replace('\u3000', ' ')
text

'第2日、12番でアプローチショットを放つ細野勇策。通算16アンダーで首位＝東建多度CC名古屋 東建ホームメイト・カップ第2日（31日・三重県東建多度CC名古屋＝7062ヤード、パー71）20歳でツアー未勝利の細野勇策が3位から11バーディー、1ボギーの61をマークし、通算16アンダー、126で首位に立った。61は同コース開催での大会最少スコアを1打更新。4打差で前日首位の今平周吾が続いた。\n 通算10アンダーの3位に64で回った田中裕基とマイケル・ヘンドリー（ニュージーランド）がつけ、さらに1打差で石川遼らが並んだ。金谷拓実は首位から71と伸ばせず7アンダーの12位に後退。昨年優勝の香妻陣一朗は3アンダーの50位。2アンダーまでの79人が決勝ラウンドに進んだ。'

In [249]:
from gpt_index import StringIterableReader, GPTTreeIndex

documents = StringIterableReader().load_data([text])
index = GPTSimpleVectorIndex.from_documents(documents)

In [280]:
index.query("<question_text>?")

Response(response='\n細野勇策は、第2日の試合で通算16アンダーで首位に立った。', source_nodes=[NodeWithScore(node=Node(text='第2日、12番でアプローチショットを放つ細野勇策。通算16アンダーで首位＝東建多度CC名古屋 東建ホームメイト・カップ第2日（31日・三重県東建多度CC名古屋＝7062ヤード、パー71）20歳でツアー未勝利の細野勇策が3位から11バーディー、1ボギーの61をマークし、通算16アンダー、126で首位に立った。61は同コース開催での大会最少スコアを1打更新。4打差で前日首位の今平周吾が続いた。\n 通算10アンダーの3位に64で回った田中裕基とマイケル・ヘンドリー（ニュージーランド）がつけ、さらに1打差で石川遼らが並んだ。金谷拓実は首位から71と伸ばせず7アンダーの12位に後退。昨年優勝の香妻陣一朗は3アンダーの50位。2アンダーまでの79人が決勝ラウンドに進んだ。', doc_id='948fd1d1-782b-4593-bfa5-bdd056e6bec9', embedding=None, doc_hash='e28622816e464dca564852727a35e9fd913f41db896c4d57d83e345f1db0d079', extra_info=None, node_info={'start': 0, 'end': 334}, relationships={<DocumentRelationship.SOURCE: '1'>: '57cbe2c9-a194-478c-817b-a27fb962a1b1'}), score=0.6840508248962084)], extra_info=None)

In [273]:
index.query("次の問いに日本語で答えてください。単語で出力してください。\n\n誰が首位ですか?").response.strip()

'細野勇策'

In [251]:
index.query("要約してください").response.strip()

'細野勇策が3位から11バーディー、1ボギーの61で通算16アンダー、126で首位に立ち、大会最少スコアを1打更新した。田中裕基とマイケル・ヘンドリーが3位につけ、石川遼らが1打差で並んだ。金谷拓実は7アンダーの12位に後退し、香妻陣一朗は3アンダーの50位であった。79人が2アンダーまでで決勝ラウンドに進んだ。'

## 解答候補を抽出する

In [252]:
!pip install -qU spacy ja_ginza

In [253]:
import spacy

nlp=spacy.load('ja_ginza')

In [278]:
summary_text = index.query("要約してください").response.strip()  # 本来は、記事そのままでよいだろう

In [279]:
doc = nlp(summary_text)
doc

細野勇策が3位から11バーディー、1ボギーの61で通算16アンダー、126で首位に立ち、大会最少スコアを1打更新した。今平周吾が4打差で2位、田中裕基とマイケル・ヘンドリー（ニュージーランド）が3位、石川遼らが1打差で4位となり、金谷拓実は7アンダーの12位に後退した。香妻陣一朗は3アンダーの50位で、2アンダーまでの79人が決勝ラウンドに進んだ。

In [257]:
[ (e.text, e.label_) for e in doc.ents ]

[('細野勇策', 'Person'),
 ('3位', 'Rank'),
 ('11バーディー', 'N_Organization'),
 ('1ボギー', 'Period_Day'),
 ('61', 'Numex_Other'),
 ('16アンダー', 'N_Product'),
 ('126', 'Numex_Other'),
 ('首位', 'Rank'),
 ('1打', 'N_Event'),
 ('田中裕基', 'Person'),
 ('マイケル・ヘンドリー', 'Person'),
 ('ニュージーランド', 'Country'),
 ('10アンダー', 'N_Product'),
 ('3位', 'Rank'),
 ('石川遼', 'Person'),
 ('1打', 'N_Event'),
 ('4位', 'Rank'),
 ('金谷拓実', 'Person'),
 ('71', 'Numex_Other'),
 ('7アンダー', 'N_Product'),
 ('12位', 'Rank'),
 ('香妻', 'Person'),
 ('3アンダー', 'N_Product'),
 ('50位', 'Rank'),
 ('2アンダー', 'N_Product'),
 ('79人', 'N_Person')]

## 関連情報を得る

ひとまず Wikipedia 日本語版の記事サマリーを用いる。ある解答候補について関連情報を得たら、そちらに基づいて質問を生成してもよい。

- [PythonでWikipediaの情報を取得する | 分析ノート](https://analytics-note.xyz/programming/python-wikipedia/)
- [wikipedia · PyPI](https://pypi.org/project/wikipedia/)

In [258]:
import wikipedia
wikipedia.set_lang("ja")

In [259]:
for e in doc.ents:
  if e.label_ in ['Person', 'Company', 'Country']:
    print( e.text,  wikipedia.search(e.text, results=3))

細野勇策 ['さすがの猿飛', '日本の写真家一覧', '岐阜県立岐阜高等学校']
田中裕基 ['Over The Top (バンド)', '田中裕子', '田中裕二 (お笑い芸人)']
マイケル・ヘンドリー ['マイケル・ジャクソン', 'ジミ・ヘンドリックス', 'マイケル・ランドウ']
ニュージーランド ['ニュージーランド', 'ニュージーランド軍', 'ニュージーランド・レコード産業協会']
石川遼 ['石川遼', '石川遼スペシャル RESPECT 〜ゴルフを愛する人々へ〜', 'ゴルフ日本シリーズ']
金谷拓実 ['金谷拓実', 'インターナショナル・マネジメント・グループ', 'ダンロップフェニックストーナメント']
香妻 ['香妻琴乃', '香妻陣一朗', 'ゴルファー一覧']


「さすがの猿飛」は細野不二彦。おそらく間違い。

In [260]:
rel_urls = []
for e in doc.ents:
  if e.label_ in ['Person', 'Company', 'Country']:
    wp_ents = wikipedia.search(e.text, results=3)
    for wp in wp_ents:
      wp_page = wikipedia.page(wp)
      if e.text in wp_page.summary: # wp_page.contentでもよいが
        rel_urls.append(wp_page.summary)

rel_urls

['ニュージーランド\nNew Zealand （英語）Aotearoa （マオリ語）\n\n国の標語：なし 1\n国歌：God Defend New Zealand（英語）Aotearoa（マオリ語）神よニュージーランドを守り給え（アオテアロア）God Save the King（英語）神よ国王を護り賜え \n\n注1：かつての標語は、「Onward（英語：さらに先へ）」\n注2：チャタム諸島は、ニュージーランド本土よりも45分進んでいる。ニュージーランド（英語: New Zealand、マオリ語: Aotearoa）は、南西太平洋のオセアニアのポリネシアに位置する立憲君主制国家。首都はウェリントンで、最大の都市はオークランドである。\n島国であり、二つの主要な島と、多くの小さな島々からなる。北西に2,000km離れたオーストラリア大陸（オーストラリア連邦）と対する。南方の南極大陸とは2,600km離れている。北はトンガ、ニューカレドニア、フィジーがある。イギリス連邦加盟国であり、英連邦王国の一国となっている。また、ニュージーランド王国を構成する最大の主体地域である。',
 'ニュージーランド国防軍（ニュージーランドこくぼうぐん、英語: New Zealand Defence Force マオリ語: Te Ope Kātua o Aotearoa）は、ニュージーランドにおける国防組織。陸海空の三軍からなる。',
 'ニュージーランド・レコード産業協会（ニュージーランド・レコードさんぎょうきょうかい、Recorded Music NZ）は、ニュージーランドでの音楽販売に関わる音楽プロデューサー、販売者、音楽家らによる非営利の業界団体。RIANZ は、ニュージーランドで営業する音楽レーベルならば参加可能となっているが、米国と英国の「ビッグ4」（EMI、ソニー、ユニバーサル、ワーナー）が全体の 4/5 を占めて運営を取り仕切っている。ビッグ4以外に53の企業が参加している。',
 '『石川遼スペシャル RESPECT 〜ゴルフを愛する人々へ〜』（いしかわりょうスペシャル リスペクト ゴルフをあいするひとびとへ）は、2010年4月4日から2012年3月31日まで、テレビ東京系列で放送されていたゴルフトーク番組。石川遼の冠番組。\n\n',
 '香妻 琴乃（こうづま ことの

In [261]:
if False:
  documents_rel = SimpleWebPageReader().load_data(rel_urls)
  index_rel = GPTSimpleVectorIndex.from_documents(documents_rel)

In [262]:
from gpt_index import StringIterableReader, GPTTreeIndex

documents_rel = StringIterableReader().load_data(rel_urls)
index_rel = GPTSimpleVectorIndex.from_documents(documents_rel)

## 質問生成 四択問題

In [263]:
index.query("あなたは有能なクイズ作家です。細野勇作が正解になる四択クイズを作ってください。").response.strip()

'Q. 第2日の大会で首位に立ったのは誰でしたか？\nA. 1. 今平周吾  2. 細野勇策  3. 田中裕基  4. マイケル・ヘンドリー'

In [264]:
index.query("あなたは有能なクイズ作家です。細野勇作が正解になる四択クイズを作ってください。3つ作ってください").response.strip()

'1. 細野勇策はどのツアーで未勝利でしたか？\nA. PGAツアー\nB. プロゴルフツアー\nC. ワールドゴルフツアー\nD. ヨーロッパツアー\n\n2. 細野勇策が獲得した大会最少スコアは何打でしたか？\nA. 62\nB. 63\nC. 64\nD. 61\n\n3. 細野勇策が首位に立った大会の通算スコアは何でしたか？\nA. 15アンダー\nB. 16アンダー\nC. 17アンダー\nD. 18アンダー'

## 検証: 質問＋各選択肢が成立するか

正例負例それぞれで成立不成立を問い直す

In [265]:
query = """あなたは有能なクイズ作家です。次の質問と回答のペアは適切ですか。YES/NOで答えてください。

質問: 細野勇策が通算16アンダー首位となったのはどこで行われたゴルフ大会でしたか？
回答: 東建ホームメイト・カップ
"""

In [266]:
index.query(query).response.strip()

'YES'

## 導入文の生成

In [267]:
query_template = """あなたは有能なクイズ作家です。次の質問と回答のペアに対して、クイズとして適切な導入文を付与してください。導入文には、質問の内容を含みません。導入文は疑問文ではありません。導入文には、解答の内容を含みません。 5つ、作成してください。

質問: {question}
回答: {answer}
"""

In [268]:
question = "細野勇策が通算16アンダー首位となったのはどこで行われたゴルフ大会でしたか？"
answer = "東建ホームメイト・カップ"

In [269]:
query = query_template.format(question=question, answer=answer)

query

'あなたは有能なクイズ作家です。次の質問と回答のペアに対して、クイズとして適切な導入文を付与してください。導入文には、質問の内容を含みません。導入文は疑問文ではありません。導入文には、解答の内容を含みません。 5つ、作成してください。\n\n質問: 細野勇策が通算16アンダー首位となったのはどこで行われたゴルフ大会でしたか？\n回答: 東建ホームメイト・カップ\n'

In [270]:
index.query(query).response.strip().replace('\n','')

'1. 東建ホームメイト・カップで細野勇策が通算16アンダーで首位を獲得しました。2. 細野勇策が通算16アンダーで首位を獲得したのは、東建ホームメイト・カップでした。3. 東建ホームメイト・カップで細野勇策が通算16アンダーで首位を獲得しました。4. 細野勇策が東建ホームメイト・カップで通算16アンダーで首位を獲得しました。5. 東建ホームメイト・カップで細'

In [271]:
index_rel.query(query).response.strip().replace('\n','')

'1. 細野勇策がゴルフ大会で何を達成したのかご存知ですか？2. 東建ホームメイト・カップで細野勇策が何を達成したのでしょうか？3. 細野勇策がゴルフ大会で何を達成したか、ご存知ですか？4. 細野勇策がゴルフ大会で何を達成したのか、ご存知ですか？5. 東建ホームメイト・カップで細野勇策が何を達成したのか、ご存知ですか？'

## 導入文と質問・解答をペアにしてすわりがよいかを評価
## クイズの難しさの評価
## クイズとしての面白さの評価


TBD

----


In [272]:
index.query("""あなたはプロのクイズ作家です。特に四択問題を作ることを得意としています。 以下のテーマでクイズを作ってください。

テーマ：ボーイング737MAX""").response.strip()

'Q. ボーイング737MAXは何番目の737シリーズですか？\n\nA. ボーイング737MAXは737シリーズの9番目です。'

In [139]:
index.query("""あなたはプロのクイズ作家です。特に四択問題を作ることを得意としています。 以下のテーマで四択クイズを作ってください。

テーマ：細野勇策""").response.strip()

'Q. 細野勇策が通算16アンダー首位を獲得したのはどこでしたか？\n\nA. 東建ホームメイト・カップ第2日（31日・三重県東建多度CC名古屋＝7062ヤード、パー71）'

In [140]:
index.query("""あなたはプロのクイズ作家です。特に四択問題を作ることを得意としています。 以下のテーマで四択クイズを作ってください。

テーマ：東建""").response.strip()

'Q. 東建ホームメイト・カップ第2日（31日・三重県東建多度CC名古屋＝7062ヤード、パー71）で首位を獲得したのは誰でしたか？\n\nA. 共同通信が報じたところによると、細野勇策が東建ホームメイト・カップ第2日（31日・三重県東建多度CC名古屋＝7062ヤード、パー71）で首位を獲得したとのことです。'

In [216]:
index.query("""あなたはプロのクイズ作家です。特に四択問題を作ることを得意としています。 以下のテーマで四択クイズを作ってください。3つ作ってください。

テーマ：東建""").response.strip()

'1. 東建ホームメイト・カップはどこで開催されましたか？\nA. 東京\nB. 三重県東建多度CC名古屋\nC. 大阪\nD. 北海道\n\n2. 東建ホームメイト・カップの第2日で、細野勇策が通算何アンダーで首位になりましたか？\nA. 10アンダー\nB. 12アンダー\nC. 14アンダー\nD. 16アンダー\n\n3. 東建ホームメイト・カップの第2日で、細野勇策が首位になった時の年齢は何歳でしたか？'

In [217]:
index_rel.query("""あなたはプロのクイズ作家です。特に四択問題を作ることを得意としています。 以下のテーマで四択クイズを作ってください。3つ作ってください。

テーマ：東建""").response.strip()

'1. 東建ホームメイトカップは何月に開催されるか？\nA. 1月\nB. 2月\nC. 3月\nD. 4月\n\n2. 東建ホームメイトカップは何のトーナメントであるか？\nA. テニス\nB. バスケットボール\nC. バレーボール\nD. ゴルフ\n\n3. 東建ホームメイトカップは誰が主催しているか？\nA. 日本ゴルフツアー\nB. 東建コーポレーション\nC. 日本テニス協会\nD. 日本バスケットボール協会'

あなたはプロのクイズ作家です。特に四択問題を作ることを得意としています。
以下のテーマでクイズを作ってください。

テーマ：ボーイング737MAX



ボーイング737MAXはエンジン火災の危険性の問題で一時的に運行停止になりましたか? YES/NOで答えてください

次の質問と回答のペアは正しいですか? YES/NOで答えてください。理由もお願いします。

質問: ボーイング737MAXはどのような問題で一時的に運行停止になりましたか？
回答: MCASシステムの不具合による墜落事故

次の質問と回答のペアが正しくなるよう、新しい質問を作ってください。

質問: ボーイング737MAXはどのような問題で永久に運行停止になりましたか？
回答: MCASシステムの不具合による墜落事故

次の質問と回答のペアに対して、クイズとして適切な導入文を付与してください。導入文には、質問の内容を含みません。導入文には、解答の内容を含みません。 5つ、作成してください。